In [1]:
import wlogs as wl
import pandas as pd
from datetime import datetime, timedelta

In [2]:
def convert_time(epoch_time_ms):
    return datetime.fromtimestamp(float(epoch_time_ms)/1000)


def convert_time_str(epoch_time_ms, fmt = "%d/%m/%Y %H:%M:%S"):
    return datetime.fromtimestamp(float(epoch_time_ms)/1000).strftime(fmt)

In [3]:
def compose_report_data_df(report_data: dict):
    report_code = report_data['code']

    actors = pd.DataFrame(report_data['masterData']['actors']).set_index(keys='id').rename(columns={'name': 'player_name'})
    actors = actors[actors['subType']!='Unknown']
    actors['report_code'] = report_code

    fights = pd.DataFrame(report_data['fights']).set_index(keys='id').rename(columns={'name': 'boss_name'})
    fights = fights[fights['size'].notna()]
    fights['report_code'] = report_code

    report_data_df = fights.merge(actors, on='report_code')

    add_metadata = {'report_date': convert_time_str(report_data['startTime'], fmt = "%d/%m/%Y"),
                    'report_start': convert_time(report_data['startTime']),
                    'report_end': convert_time(report_data['endTime'])}
    for key, value in add_metadata.items():
        report_data_df[key] = value

    return report_data_df

In [4]:
reports_data = wl.load_reports_data()

reports_df = []
for report_data in reports_data.values():
    report_data_df = compose_report_data_df(report_data)
    reports_df.append(report_data_df)

progress_data = pd.concat(reports_df)

C:\Users\matth\AppData\Local\Temp\ipykernel_24560\685193825.py:8: FutureWarning: In a future version, object-dtype columns with all-bool values will not be included in reductions with bool_only=True. Explicitly cast to bool dtype instead.
  progress_data = pd.concat(reports_df)


In [5]:
ulduar_boss_hm = ['Flame Leviathan', 'XT-002 Deconstructor', 'The Assembly of Iron', 'Hodir', 'Thorim', 'Freya', 'Mimiron', 'General Vezax', 'Yogg-Saron']

raiders = ['Åkodah', 'Arkye', 'Bobsatan', 'Codd', 'Cynyancat', 'Drep', 'Ehunkel', 'Eskablip', 'Frallak', 'Gogø', 'Gymble', 'Kekozolives', 'Kienlim', 'Krohl', 'Landrîî', 'Madess', 'Melvira', 'Memø', 'Narn', 'Neandis', 'Oréliana', 'Padreleon', 'Pahâthata', 'Sarmath', 'Selbie', 'Selemental', 'Sharigan', 'Takamaka', 'Teraxio', 'Tonsurédeouf', 'Urgoz', 'Vrôum', 'Xargøs']

progress_ulduar_10HM = progress_data[(progress_data['kill'] == True) & (progress_data['size'] == 10) & (~progress_data['hardModeLevel'].isna()) & ((progress_data['hardModeLevel'] != 0) | ((progress_data['report_code']=='L1hbC8dX2c7wDKx6') & (progress_data['boss_name']=='Hodir'))) & (progress_data['boss_name'].isin(ulduar_boss_hm)) & (progress_data['player_name'].isin(raiders))].groupby(by=['boss_name', 'player_name'])['report_date'].min().reset_index()

In [6]:
boss_killed = progress_ulduar_10HM['boss_name'].unique().tolist()
boss_killed_ordered = [boss for boss in ulduar_boss_hm if boss in boss_killed]

first_kill_ulduar_10HM = progress_ulduar_10HM.pivot(index='player_name', columns='boss_name', values='report_date')[boss_killed_ordered]

In [7]:
wl.save_df_to_csv(first_kill_ulduar_10HM, 'first_kill_ulduar10HM')